In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, matthews_corrcoef, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData
import importlib

importlib.reload(ML)
importlib.reload(PrepareData)

<module 'PrepareData' from 'C:\\Users\\Adam\\source\\repos\\WeatherPrediction\\WeatherPredictionProject\\PrepareData.py'>

In [2]:
# method 'teach_models' takes the data and tech all models with it

def teach_models(X_train, y_train):
    models=[]

    X_train, y_train = SMOTETomek().fit_resample(X_train, y_train)

    models.append(ML.NeuralNetworks(X_train, y_train))
    models.append(ML.KNN(X_train, y_train))
    models.append(ML.DecisionTree(X_train, y_train))
    models.append(ML.LinearRegressionModel(X_train, y_train))
    models.append(ML.LogisticRegressionModel(X_train, y_train))
    models.append(ML.RandomForest(X_train, y_train))
    models.append(ML.SupportVectorMachines(X_train, y_train))
    models.append(ML.SupportVectorRegression(X_train, y_train))
    models.append(ML.TreeGradientBoosting(X_train, y_train))
    models.append(ML.DummyModel(X_train, y_train))

    return models

In [3]:
# method 'score_models' calculate scores to many statistics for all models separately

def score_models(models, X_test, y_test):
    results=[]
    for model in models:
        scores={}
        y_pred=model.predict(X_test)>0.5
        scores['Model']=model.__str__()
        scores['Accuracy']=accuracy_score(y_test, y_pred)
        scores['F1_score']=f1_score(y_test, y_pred)
        scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred)
        scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred)
        scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred))
        scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred)
        scores['Confusion_matrix']=confusion_matrix(y_test, y_pred)

        results.append(scores)
    return results

In [4]:
# method 'score_merged_models' calculate scores to many statistics for selected models as merged object

def score_merged_models(models, X_test, y_test):
    merged_scores={}
    results=pd.DataFrame()
    num=1
    for model in models:
        column="Model_"+str(num)
        results[column]=list(model.predict(X_test)>0.5)
        num+=1

    results=results.astype(np.int32)

    results["Sum"]=results.sum(axis=1)/len(models)>0.5

    y_pred_merged=results["Sum"].values>0.5

    merged_scores['Model']='Merged model'
    merged_scores['Accuracy']=accuracy_score(y_test, y_pred_merged)
    merged_scores['F1_score']=f1_score(y_test, y_pred_merged)
    merged_scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred_merged)
    merged_scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred_merged)
    merged_scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred_merged))
    merged_scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred_merged)
    merged_scores['Confusion_matrix']=confusion_matrix(y_test, y_pred_merged)

    return merged_scores

In [5]:
# cell with data read, split it into train and test parts and teach the models

df=PrepareData.get_data()

df=df.dropna()

y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

models=teach_models(X_train, y_train)

Epoch 1/25
327/327 [==============================] - 2s 4ms/step - loss: 0.1194 - f1: 0.6008 - accuracy: 0.5567 - val_loss: 0.1624 - val_f1: 0.1498 - val_accuracy: 0.2042
Epoch 2/25
327/327 [==============================] - 1s 2ms/step - loss: 0.1097 - f1: 0.6557 - accuracy: 0.5873 - val_loss: 0.1561 - val_f1: 0.1513 - val_accuracy: 0.6282
Epoch 3/25
327/327 [==============================] - 1s 2ms/step - loss: 0.1088 - f1: 0.6253 - accuracy: 0.5797 - val_loss: 0.1564 - val_f1: 0.1548 - val_accuracy: 0.5955
Epoch 4/25
327/327 [==============================] - 1s 2ms/step - loss: 0.1073 - f1: 0.6528 - accuracy: 0.5984 - val_loss: 0.1551 - val_f1: 0.1547 - val_accuracy: 0.6185
Epoch 5/25
327/327 [==============================] - 1s 2ms/step - loss: 0.1052 - f1: 0.6649 - accuracy: 0.6181 - val_loss: 0.1512 - val_f1: 0.1519 - val_accuracy: 0.6621
Epoch 6/25
327/327 [==============================] - 1s 2ms/step - loss: 0.1036 - f1: 0.6725 - accuracy: 0.6283 - val_loss: 0.1462 - val_f1

In [6]:
# simple usage of 'score_models' method

scores=score_models(models,X_test,y_test)
scores[0]

63/63 [==============================] - 0s 1ms/step


{'Model': '<keras.engine.sequential.Sequential object at 0x00000202F87BB700>',
 'Accuracy': 0.7023394723743156,
 'F1_score': 0.7632620744259699,
 'Matthwes_correlation_coefficient_(MCC)': 0.363119919043178,
 'Mean_squared_error_(MSE)': 0.2976605276256844,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5455827413194853,
 'Mean_absolute_error_(MAE)': 0.2976605276256844,
 'Confusion_matrix': array([[447, 320],
        [278, 964]], dtype=int64)}

In [7]:
# simple usage of 'score_merged_models' method

scores=score_merged_models(models,X_test,y_test)
scores

63/63 [==============================] - 0s 2ms/step


{'Model': 'Merged model',
 'Accuracy': 0.6978596316575411,
 'F1_score': 0.7398199742820403,
 'Matthwes_correlation_coefficient_(MCC)': 0.38775259450861216,
 'Mean_squared_error_(MSE)': 0.30214036834245894,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5496729649004569,
 'Mean_absolute_error_(MAE)': 0.30214036834245894,
 'Confusion_matrix': array([[539, 228],
        [379, 863]], dtype=int64)}

In [8]:
# The below algorithm allow determining which set of models is best (needs ~10 min)

best_score=0
best_models=[]
num=0

import itertools
for L in range(2,len(models) + 1):
    for subset in itertools.combinations(models[:-1], L):
        merged_scores=score_merged_models(list(subset),X_test,y_test)
        if merged_scores['F1_score']>best_score:
            best_score=merged_scores['F1_score']
            best_models=list(subset)
        if num%100==0:
            print(num)
        num+=1

print(best_score)
print(best_models)

63/63 [==============================] - 0s 1ms/step
0
63/63 [==============================] - 0s 1ms/step
100
63/63 [==============================] - 0s 1ms/step
200
63/63 [==============================] - 0s 1ms/step
300
63/63 [==============================] - 0s 1ms/step
400
63/63 [==============================] - 0s 1ms/step
500
63/63 [==============================] - 0s 1ms/step
0.7705905986339896
[<keras.engine.sequential.Sequential object at 0x00000202F87BB700>, DecisionTreeClassifier(), LGBMClassifier()]
